# STA 141B Data & Web Technologies for Data Analysis

### Lecture 15, 11/19/24, Natural language processing


### Last week's topics

- Tokenizing
- Tagging
- Topic Modelling

### Today's topics
- Natural Language Processing
    - Naive Bayes
    - ROC space

The goal of today is to answer the following questions:
1. How can we identify particular features of language data that are salient for classifying it?
2. How can we construct models of language that can be used to perform language processing tasks automatically?
3. What can we learn about language from these models?

### Naive Bayes Classifier

We aim to find the probability for a label. The following algorithm first uses the Bayes rule to express $P(label|features)$ in terms of $P(label)$ and $P(features|label)$, where the latter are approximated from the test data set: 
$$
P(label|features) = \frac{P(features|label)P(label)}{P(features)}
$$

As usual for discrete data, the probability in the denominator is not computed directly. Instead, the numerator is normalized. 

Now, we make the 'naive' assumption that all features are independent, given the label:
$$
P(label|features) \propto P(label) P(feature\ 1|label) \dots P(feature\ n|label)
$$

To exemplify, we will consider a simple case first, a single feature. 

In [ ]:
from nltk.corpus import names
import nltk
import random
import numpy as np

In [ ]:
names = ([(name, 'male') for name in names.words('male.txt')] + 
         [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)

In [ ]:
names[:4]

In [ ]:
def gender_features(word):
    return word[-1]

In [ ]:
gender_features("Trinity")

In [ ]:
featuresets = [({'feature': gender_features(n)}, g) for n, g in names]
train_set, test_set = featuresets[500:], featuresets[:500]

In [ ]:
train_set[:3]

First, lets estimate $P(label)$. 

In [ ]:
P_label = np.mean([True if g[1] == 'male' else False for g in train_set])
print(P_label)
print(np.mean([True if g[1] == 'female' else False for g in train_set]))

In [ ]:
def likelihood(feature, label): 
    return np.mean([True if f['feature'] == feature else False for f, y in train_set if y == label])

In [ ]:
likelihood('a', 'female')

To compute the posterior, lets establish how many features there are. 

In [ ]:
features = {f['feature'] for f, g in train_set}
len(features)

In [ ]:
#features

In [ ]:
posterior_male = {f: P_label * likelihood(f, 'male') for f in features}
posterior_female = {f: (1 - P_label) * likelihood(f, 'female') for f in features}

In [ ]:
normalization = {k: posterior_male.get(k, 0) + posterior_female.get(k, 0) for k in set(posterior_female) & set(posterior_male)}
normalization

In [ ]:
posterior_male = {f: P_label * likelihood(f, 'male') / normalization.get(f, 0) for f in features}
posterior_female = {f: (1 - P_label) * likelihood(f, 'female') / normalization.get(f, 0) for f in features}

In [ ]:
print(posterior_male.get('k')) # prob of label given feature
print(posterior_female.get('k'))

Our classifier is implemented. How well does it do? 

In [ ]:
posterior_female.get(gender_features('Esther')) 

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.prob_classify({'feature': gender_features('Esther')}).prob('female')

In [ ]:
classifier.prob_classify({'feature': gender_features('Esther')}).samples()

In [ ]:
gender_features('Esther')

Lets classify [Moby Dick](https://www.gutenberg.org/files/2701/2701-h/2701-h.htm).

In [ ]:
import re
import nltk
import numpy as np
import pandas as pd

In [ ]:
moby = nltk.corpus.gutenberg.raw("melville-moby_dick.txt")
pattern = r"(?<!,\s{1})(?:ETYMOLOGY|CHAPTER\s{1}\d+|Epilogue|EXTRACTS(?=\s*\())(?:\.*\s*).+?\s*.*[\.{1}|!{1}|\?{1}|\){1}]"
corpus = re.split(pattern, moby)
corpus.pop(0)
corpus = [re.sub(r"\s+", " ", document).lower() for document in corpus]

In [ ]:
len(corpus)

From [here](https://digitalcommons.cwu.edu/cgi/viewcontent.cgi?article=1430&context=etd#:~:text=In%20Chapters%201%2D8%2C%2010,role%20of%20%22I%22%20narrator.). 

In [ ]:
cetological = [0, 1, 26, 27, 46, 55, 57, 58, 59, 60, 61, 63, 64, 68, 75, 76, 77, 78, 80, 81, 85, 86, 87, 89, 90, 91, 93, 96, 97, 98, 102, 103, 104, 105, 106, 107]

In [ ]:
story = [i not in cetological for i in range(138)]
story = [i for i in map(lambda x: 'story' if x else 'ceto', story)]

In [ ]:
def tokenizer(document): 
    words = re.findall('\w+', document)
    words = [w[0] for a, w in nltk.pos_tag(words) if w[0] in ['N', 'V', 'J', 'R', 'P']]
    return words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(tokenizer = tokenizer) 
tfidf = vec.fit_transform(corpus)

In [ ]:
def get_row_as_dict(i): 
    row = tfidf[i,:].todense().tolist()[0]
    names = vec.get_feature_names_out()
    return {n: r for r, n in zip(row, names)}

In [ ]:
get_row_as_dict(130)

In [ ]:
featuresets = [(get_row_as_dict(i), story[i]) for i, _ in enumerate(corpus)]

In [ ]:
featuresets[0][1]

In [ ]:
#random.shuffle(featuresets)
train_set, test_set = featuresets[:90], featuresets[90:]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
print(nltk.classify.accuracy(classifier, test_set))

In [ ]:
sum([True for i in story if i=="story"]) / len(story) #benchmark accuracy

In [ ]:
fp = fn = tp = tn = 0
for test in test_set: 
    classification = classifier.classify(test[0])
    truth = test[1]
    if classification == truth and truth == 'story': tp += 1
    if classification == truth and truth != 'story': tn += 1
    if classification != truth and truth == 'story': fp += 1
    if classification != truth and truth != 'story': fn += 1

In [ ]:
confusion = pd.DataFrame([[tp, fn], [fp, tn]], 
                         columns = ['true story', 'true ceto'], index = ['class story', 'class ceto'])
confusion

We assign a chapter to be either cetological or storyline depending on the respective probability exceeding $0.5$. But this is an arbitrary decision rule.

Instead, one should consider choosing cetological chapters based on different thresholds. 

| Estimated \ True      | Success        | Failure       |
| ----------- | ----------- | ----------- |
| Success        | $n_{1|1}$, true positive   | $n_{1|2}$, false positive |
| Failure       | $n_{2|1}$, false negative  | $n_{2|2}$, true negative  |

From the values of the confusion matrix the following measures can be calculated for the evaluation of the classification model: 
 - true positive rate (TPR / sensitivity): $\frac{n_{1|1}}{n_{1|1} + n_{2|1}}$
 - false negative rate: $\frac{n_{2|1}}{n_{1|1} + n_{2|1}}$
 - false positive rate (FPR): $\frac{n_{1|2}}{n_{1|2} + n_{2|2}}$
 - true negative rate (specificity): $\frac{n_{2|2}}{n_{1|2} + n_{2|2}}$
 - positive prediction rate (precision): $\frac{n_{1|1}}{n_{1|1} + n_{1|2}}$
 - false detection rate: $\frac{n_{1|2}}{n_{1|1} + n_{1|2}}$
 - negative prediction rate (separability): $\frac{n_{2|2}}{n_{2|1} + n_{2|2}}$
 - misclassification rate: $\frac{n_{2|1}}{n_{2|1} + n_{2|2}}$

In [ ]:
confusion.iloc[0, 0] / (confusion.iloc[0, 0] + confusion.iloc[1, 0]) #TPR

In [ ]:
confusion.iloc[0, 1] / (confusion.iloc[0, 1] + confusion.iloc[1, 1]) #FPR

### ROC Plots

A ROC (receiver operating characteristics) plot plots the FPR on the x-axis onto the TPR on the y-axis. 

The point $(0,0)$ means that a model never classifies a success: $n_{1|1} = n_{1|2} = 0$. 
This way we avoid false positive errors, at the cost of never having any true positive classifications either. 
Point $(0,1)$ corresponds to a perfect classification $n_{2|1} = n_{1|2} = 0$. 

A point is *better in the ROC space* than another point, if its TPR is larger and its FPR smaller than the TPR and FPR of the other point. 

A point below the bisector $x = y$ corresponds to a point which is worse than a uniformly random classification. The bisector corresponds to a uniform random classification. 

We can use classification methods that return decision for the class (e.g., success or failure), or methods that return a probability that the observation belongs to a certain class (e.g., $\pi$). For the latter, we have to set a threshold $\tau\in[0,1]$, to retrieve a true classification. 
Changing the threshold, will change the TPR and FPR. This leads to a *ROC curve*. 

Let $\hat\pi_i\in[0,1]$ be the estimate for probability of the i-th observation is a success. 
Define $f_j$ to be the probability mass function of a random variable $p_j$, $j = 0,1$, where the realizations of $p_j$ are the values $\hat\pi_i$ that correspond to $y_i = j$. Now we can interpret the FPR and TRP as functions of the threshold $\tau$. 

$$
TPR(\tau) = \int_\tau^\infty f_1(x) dx\qquad FPR(\tau) = \int_\tau^{\infty} f_0(x) dx 
$$

This gives the ROC curve $\tau\rightarrow(FPR(\tau), TPR(\tau))$ as a function of $\tau$.

In [ ]:
def TPR(tau, classifier, featuresets, target = 'story'): 
    pi = np.array([classifier.prob_classify(chapter[0]).prob(target) for chapter in featuresets if chapter[1] == 'story'])
    norm = len([True for chapter in featuresets if chapter[1] == 'story'])
    tpr = [sum(pi >= t) / norm for t in tau]
    return tpr

In [ ]:
TPR([0], classifier, test_set)

In [ ]:
def FPR(tau, classifier, featuresets, target = 'story'): 
    pi = np.array([classifier.prob_classify(chapter[0]).prob(target) for chapter in featuresets if chapter[1] != 'story'])
    norm = len([True for chapter in featuresets if chapter[1] != 'story'])
    fpr = [sum(pi >= t) / norm for t in tau]
    return fpr

In [ ]:
FPR([0], classifier, test_set)

In [ ]:
df = pd.DataFrame({
    'tau': [0, 0.1, 0.2, 0.5, 0.9, 1],
    'tpr': TPR([0, 0.1, 0.2, 0.5, 0.9, 1], classifier, test_set),
    'fpr': FPR([0, 0.1, 0.2, 0.5, 0.9, 1], classifier, test_set)
})

In [ ]:
df

In [ ]:
import plotly.express as px

fig = px.line(df, x="fpr", y="tpr", text = 'tau')
fig.update_layout(yaxis_range=[-0.1,1.1], 
                  xaxis_range=[-0.1,1.1])
fig.update_traces(textposition="bottom right")
fig.show()

To compare different classification methods, we can use the area under the curve (AUC). 
One can show that the AUC $= P(p_0 > p_1)$. 
The larger the AUC, the better the method in classifying. 

In [ ]:
df = df.iloc[::-1]
np.trapz(df['tpr'], df['fpr']) # AUC